In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    tmp_outputs =
    [
        { ENTITY_NAME: "cache_data" },
        { ENTITY_NAME: "cache_config" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 0_data_load](./imgs/0_data_load_inteface.drawio.png)

In [ ]:
#4 write outputs
tmp_outputs = substep.tmp_outputs()

In [ ]:
print(f"{tmp_outputs.cache_data=}")

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

#### Dataset description:

The COCO (Common Objects in Context) dataset is a large-scale image recognition dataset for object detection, segmentation, and captioning tasks. It contains over 330,000 images, each annotated with 80 object categories and 5 captions describing the scene. The COCO dataset is widely used in computer vision research and has been used to train and evaluate many state-of-the-art object detection and segmentation models.

The dataset has two main parts: the images and their annotations.

The images are organized into a hierarchy of directories, with the top-level directory containing subdirectories for the train, validation, and test sets.
The annotations are provided in JSON format, with each file corresponding to a single image.
Each annotation in the dataset includes the following information:

Image file name
Image size (width and height)
List of objects with the following information: Object class (e.g., "person," "car"); Bounding box coordinates (x, y, width, height); Segmentation mask (polygon or RLE format); Keypoints and their positions (if available)
Five captions describing the scene
The COCO dataset also provides additional information, such as image super categories, license, and coco-stuff (pixel-wise annotations for stuff classes in addition to 80 object classes).

MS COCO offers various types of annotations,

Object detection with bounding box coordinates and full segmentation masks for 80 different objects
Stuff image segmentation with pixel maps displaying 91 amorphous background areas
Panoptic segmentation identifies items in images based on 80 "things" and 91 "stuff" categories
Dense pose with over 39,000 photos featuring over 56,000 tagged persons with a mapping between pixels and a template 3D model and natural language descriptions for each image
Keypoint annotations for over 250,000 persons annotated with key points such as the right eye, nose, and left hip

#### Dataset load process    
Download dataset from the internet storage    

In [ ]:
#5 load dataset 
data_url = "http://images.cocodataset.org/zips/val2017.zip"
annot_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

In [ ]:
# create folder by urls from tmp_outputs
import os.path as osp
import os
os.makedirs(tmp_outputs.cache_data)
os.makedirs(tmp_outputs.cache_config)

In [ ]:
# Download dataset
!wget {data_url} -O {osp.join(tmp_outputs.cache_data, osp.basename(data_url))}

In [ ]:
# Download annotation
!wget {annot_url} -O {osp.join(tmp_outputs.cache_data, osp.basename(annot_url))}

In [ ]:
#6 unzip dataset
!unzip -q {osp.join(tmp_outputs.cache_data, osp.basename(data_url))} -d {tmp_outputs.cache_data}
!unzip -q {osp.join(tmp_outputs.cache_data, osp.basename(annot_url))} -d {tmp_outputs.cache_data}

In [ ]:
# Touch _SUCCESS file to cache
from pathlib import Path
Path(osp.join(tmp_outputs.cache_data, '_SUCCESS')).touch()
Path(osp.join(tmp_outputs.cache_config, '_SUCCESS')).touch()

In [ ]:
!ls {tmp_outputs.cache_data}

In [ ]:
!ls {tmp_outputs.cache_config}

In [ ]:
!ls {osp.join(tmp_outputs.cache_data, "annotations")}

In [ ]:
#7 stop spark
SinaraSpark.stop_session()